### 모듈 import 하기

In [45]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
import re
import urllib.request


import nltk
from konlpy.tag import Okt
from nltk.tokenize import word_tokenize
okt = Okt()

from tqdm import tqdm
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

### 데이터 전처리
- 데이터셋 불러오기
- 원하는 데이터프레임으로 만들어주기

In [8]:
badcomm = pd.read_csv('./비속어데이터셋_전처리o.csv')
badcomm.sample(10)

,댓글,비속어여부
1548,건설교통부는 뭐냐 국토교통부지,0
719,누나 내 쥬예즤 주겅,1
579,디지기전 이름은 카라스마 렌야,0
705,야동도 야동이지만 게임이나 애니 같은 것도 빠질 수 없음진짜 일본 문화 한국인들 싹...,1
3993,어제꺼를 쳐올리네,1
684,그이상 받는건 불공평한거다,0
511,다들 속으로 똑같이 생각하면서 아닌척 정의감에 불타느척 해야 사회적 으로 형성된 공...,1
4833,상습 주작글 올리는 개새끼야,1
1941,홍어 쉑 만날 만 처올리노,1
3317,옆에서 쓴소리하는 사람이 없으니깐 저러지 박근혜도 옆에서 쓴소리하면 보지식 기분나빵...,1


### 토큰화 - 불용어 제거

In [9]:
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','을','으로','자','에','와','한','하다','부터']

In [10]:
# 데이터셋 분리

text = badcomm['댓글'] # 시리즈 객체로 저장
score = badcomm['비속어여부']

### train, test 데이터 분리

In [11]:
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(text, score , test_size=0.2, random_state=0)
print(len(train_x), len(train_y), len(test_x), len(test_y))

5415 5415 1354 1354


In [12]:
train_x

1833                         딱봐도 남이 인증한거 파온거고 본인거라고도 안했는데
6204                                                  젖꼮찌
4727    리얼로 성정체성에 혼란느껴서 남자인데 남자좋아하는 게이는 전체 1쯤 될듯 근데 인간...
4456                                    개일베잡짓거리하다 니인생종친거다
2876                                    편돌이주제에   누굴 평가해 거
                              ...                        
4931                                                  부랄탁
3264                                               섹스자지보지
1653                                                   팝핀
2607                            민주당 드루킹댓글조작알바 새끼들 정신 못차렸네
2732                디지털이 발전하면서 예견됐던 일이지 근데 여자가 먼저 찍자고 하던데
Name: 댓글, Length: 5415, dtype: object

In [13]:
# 형태소 분석기를 사용하여 토큰화를 하면서 불용어를 제거하여 X_train에 저장

X_train = []
for sentence in tqdm(train_x):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화, 동사 기본형으로 변경
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    X_train.append(stopwords_removed_sentence)

100%|██████████| 5415/5415 [00:15<00:00, 340.01it/s]


In [14]:
print(X_train[:3])

[['딱', '보다', '남', '인증', '거', '파', '오다', '보다', '안'], ['젖꼮찌'], ['리얼', '로', '성정체성', '혼란', '느끼다', '남자', '인데', '남자', '좋아하다', '게이', '전체', '1', '쯤', '되다', '근데', '인간', '양성애자', '라', '고함']]


In [15]:
# test 데이터도 형태소 분석기를 사용하여 토큰화를 하면서 불용어를 제거하여 X_test에 저장

X_test = []
for sentence in tqdm(test_x):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화, 동사 기본형으로 변경
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    X_test.append(stopwords_removed_sentence)

100%|██████████| 1354/1354 [00:02<00:00, 475.83it/s]


In [16]:
print(X_test[:5])

[['즐', '라도', '사기꾼', '출신', '이라', '던데'], ['리더', '말', '그렇다', '박쥐', '없다', '코로나', '그거', '잡기', '힘들다', '꽈', '래야', '되다', '보지'], ['븅신', '보', '빨다', '나', '치다', '먹다', '이기'], ['똑같이', '해주다', '쫌'], ['저', '거', '황해', '하정우', '룩', '아니다']]


### 정수 인코딩

In [17]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train) #fit으로 tokenization 적용해주기
print(tokenizer.word_index)

{'있다': 1, '보다': 2, '아니다': 3, '되다': 4, '없다': 5, '새끼': 6, '다': 7, '같다': 8, '로': 9, '안': 10, '에서': 11, '아': 12, '고': 13, '못': 14, '만': 15, '나': 16, '그렇다': 17, '저': 18, '말': 19, '이다': 20, '거': 21, '그': 22, '임': 23, '먹다': 24, '존나': 25, '사람': 26, '개': 27, '게': 28, '니': 29, '야': 30, '들다': 31, '하고': 32, '돈': 33, '가다': 34, '지': 35, '때': 36, '진짜': 37, '좋다': 38, '맞다': 39, '그냥': 40, '내': 41, '적': 42, '네': 43, '더': 44, '뭐': 45, '병신': 46, '면': 47, '생각': 48, '라': 49, '인': 50, '년': 51, '일': 52, '애': 53, '받다': 54, '치다': 55, '왜': 56, '오다': 57, '알다': 58, '것': 59, '용접': 60, '인데': 61, '한테': 62, '나오다': 63, '놈': 64, '않다': 65, '기': 66, '근데': 67, '많다': 68, '글': 69, '좆': 70, '함': 71, '지다': 72, '이나': 73, '씹다': 74, '너': 75, '서': 76, '냐': 77, '씨발': 78, '정도': 79, '주다': 80, '사': 81, '이냐': 82, '여자': 83, '쓰다': 84, '랑': 85, '나다': 86, '한국': 87, '하': 88, '살다': 89, '모르다': 90, '1': 91, '일본': 92, '공부': 93, '소리': 94, '지랄': 95, '일베': 96, '나라': 97, '지금': 98, '수': 99, '많이': 100, '까지': 101, '시발': 102, '보고': 103, '딱': 104, '안되다':

In [18]:
# 빈도수가 낮은 단어들은 자연어 처리에서 배제
# 등장 빈도수가 1회인 단어들이 이 데이터에서 얼만큼의 비중을 차지하는지 확인

threshold = 2
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0  # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0  # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0  # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value 

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1 # 빈도수가 2보다 작은 단어 개수 + 1
        rare_freq = rare_freq + value # 빈도수가 2보다 작은 단어들의 빈도수 총합

        
print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)


단어 집합(vocabulary)의 크기 : 10095
등장 빈도가 1번 이하인 희귀 단어의 수: 5669
단어 집합에서 희귀 단어의 비율: 56.15651312530956
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 11.49595441364346


In [19]:
# 등장 빈도수가 2이하인 단어들의 수를 제외한 단어의 개수를 단어 집합의 최대 크기로 제한

# 전체 단어 개수 중 빈도수 2이하인 단어는 제거.
# 0번 패딩 토큰을 고려하여 + 1

# 등장 빈도가 1회 이하인 단어들은 자연어 처리에서 별로 중요하지 않을 듯하여 정수인코딩 할때 없애줄것

vocab_size = total_cnt - rare_cnt + 1
print('단어 집합의 크기 :',vocab_size)

단어 집합의 크기 : 4427


In [20]:
# 케라스 토크나이저의 인자로 넘겨주고 텍스트 시퀀스를 정수 시퀀스로 변환
# 높은 정수가 부여된 단어들은 등장 빈도수가 매우 낮다
# 빈도수가 낮은 단어들은 자연어 처리에서 배제

tokenizer = Tokenizer(vocab_size) 
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [21]:
# 정수 인코딩이 잘 되었나?
print(X_train[:5])

[[104, 2, 171, 522, 21, 2182, 57, 2, 10], [], [1428, 9, 2906, 428, 117, 61, 117, 188, 118, 855, 91, 765, 4, 67, 156, 49, 696], [27, 96, 455, 949, 88, 7, 29, 266, 552, 55], [553, 1429, 599, 2907, 1430, 21]]


In [22]:
print(train_y)

1833    0
6204    1
4727    0
4456    1
2876    1
       ..
4931    1
3264    1
1653    0
2607    1
2732    0
Name: 비속어여부, Length: 5415, dtype: int64


In [23]:
print(test_y)

1914    0
3433    0
1632    1
4906    0
2323    0
       ..
5872    1
5548    0
1624    1
30      0
6201    1
Name: 비속어여부, Length: 1354, dtype: int64


In [24]:
# y를 array 형식으로 저장해주기

y_train = np.array(train_y)
y_test = np.array(test_y)

In [25]:
print(y_train)
print(y_test)

[0 1 0 ... 0 1 0]
[0 0 1 ... 1 0 1]


### 패딩으로 댓글 길이 맞추기

In [26]:
print('리뷰의 최대 길이 :',max(len(review) for review in train_x))
print('리뷰의 평균 길이 :',sum(map(len, train_x))/len(train_x))

# 최대길이가 요상한 댓글이 있음

리뷰의 최대 길이 : 911
리뷰의 평균 길이 : 26.21865189289012


In [27]:
def below_threshold_len(max_len, nested_list):
  count = 0
  for sentence in nested_list:
    if(len(sentence) <= max_len):
        count = count + 1
  print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (count / len(nested_list))*100))

In [28]:
max_len = 30
below_threshold_len(max_len, X_train)

# 96 % 가 30 이하이기 때문에 최대글자수는 30으로 패딩해주는것이 좋을것같다.

전체 샘플 중 길이가 30 이하인 샘플의 비율: 96.30655586334257


In [29]:
# 모든 샘플의 길이를 30으로 맞춘다 - 이때 정수인코딩한 데이터 사용!
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

In [42]:
print(X_train)

[[   0    0    0 ...   57    2   10]
 [   0    0    0 ...    0    0    0]
 [   0    0    0 ...  156   49  696]
 ...
 [   0    0    0 ...    0    0    0]
 [   0    0    0 ...  307   14  898]
 [   0    0    0 ...   83 1030  221]]


### LSTM으로 분류하기
#### 하이퍼파라미터
- 임베딩 벡터 차원 = 100
- 은닉 상태 크기 = 128
- 배치 크기 = 64
- 에포크 = 15

In [44]:
!nvidia-smi

Wed May  4 06:30:01 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.103.01   Driver Version: 470.103.01   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  On   | 00000001:00:00.0 Off |                    0 |
| N/A   28C    P0    31W / 250W |    497MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [95]:
# 두 개의 선택지 중 하나를 예측하는 이진 분류 문제를 수행하는 모델    

from tensorflow.python.keras.layers import Embedding, Dense, LSTM
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.models import load_model
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint # 조기종료

# embedding_dim = 100
# hidden_units = 128

embedding_dim = 100
hidden_units = 128

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(LSTM(hidden_units))
model.add(Dense(1, activation='sigmoid')) # 로지스틱 회귀를 사용해야 하기 때문에 sigmoid

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)
mc = ModelCheckpoint('best_model_v0.0.16.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)
# mc = ModelCheckpoint('best_model_v0.0.12.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc']) # 훈련을 모니터링 하기 위한 지표 metrics=acc

In [96]:

history = model.fit(X_train, y_train, epochs=15, callbacks=[es, mc], batch_size=4, validation_split=0.2)

Epoch 1/15
1083/1083 [==============================] - 48s 43ms/step - loss: 0.6019 - acc: 0.6969 - val_loss: 0.4967 - val_acc: 0.7784

Epoch 00001: val_acc improved from -inf to 0.77839, saving model to best_model_v0.0.16.h5
Epoch 2/15
1083/1083 [==============================] - 46s 42ms/step - loss: 0.4487 - acc: 0.8105 - val_loss: 0.4696 - val_acc: 0.8061

Epoch 00002: val_acc improved from 0.77839 to 0.80609, saving model to best_model_v0.0.16.h5
Epoch 3/15
1083/1083 [==============================] - 46s 42ms/step - loss: 0.3674 - acc: 0.8497 - val_loss: 0.4889 - val_acc: 0.7959

Epoch 00003: val_acc did not improve from 0.80609
Epoch 4/15
1083/1083 [==============================] - 45s 42ms/step - loss: 0.3295 - acc: 0.8696 - val_loss: 0.4604 - val_acc: 0.8061

Epoch 00004: val_acc did not improve from 0.80609
Epoch 5/15
1083/1083 [==============================] - 43s 39ms/step - loss: 0.2965 - acc: 0.8862 - val_loss: 0.5210 - val_acc: 0.7913

Epoch 00005: val_acc did not imp

In [138]:
# 모델 정확도 테스트
loaded_model = load_model('best_model_v0.0.16.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

43/43 [==============================] - 1s 10ms/step - loss: 0.4682 - acc: 0.8013

 테스트 정확도: 0.8013


In [79]:
model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     (None, None, 100)         442700    
_________________________________________________________________
lstm_13 (LSTM)               (None, 64)                42240     
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 65        
Total params: 485,005
Trainable params: 485,005
Non-trainable params: 0
_________________________________________________________________


### 모델 사용해서 댓글 비속어 포함여부 예측해보기

In [111]:
def sentiment_predict(new_sentence):
    new_sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', '', str(new_sentence))
    new_sentence = okt.morphs(new_sentence, stem=True) # 토큰화
    new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거
    encoded = tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩
    pad_new = pad_sequences(encoded, maxlen = max_len) # 패딩
    score = float(loaded_model.predict(pad_new)) # 예측
    if(score > 0.5):
        print("{:.2f}% 확률로 비속어가 포함된 댓글입니다.\n".format(score * 100))
    else:
        print("{:.2f}% 확률로 일반 댓글입니다.\n".format((1 - score) * 100))

In [210]:
sentiment_predict('쓰레기들은 휴지통으로')

'96.91% 확률로 비속어가 포함된 댓글입니다.'

In [211]:
sentiment_predict('미쳤나 진짜 시발')

'99.48% 확률로 비속어가 포함된 댓글입니다.'

In [212]:
sentiment_predict('진주네요...여기 동네에 잠시 살았는데...여기 주차 장난아닙니다.....그냥 욕만 나오죠...')

'67.83% 확률로 일반 댓글입니다.'

In [213]:
sentiment_predict('애새끼가 초딩도 아니고')

'98.97% 확률로 비속어가 포함된 댓글입니다.'

In [205]:
sentiment_predict('성형후 사진 원래 포샵 존나 하는거 아님? 실제로 보면 오른쪽같이 안생겼을듯')

'97.14% 확률로 비속어가 포함된 댓글입니다.'

In [214]:
sentiment_predict('내말이 ㅋㅋ 아직 세상물정을 잘 모르는것 같더라...사회생활이 적었나..')

'91.17% 확률로 일반 댓글입니다.'

In [220]:
sentiment_predict('집가서 마라탕 먹자')

'46.38% 확률로 비속어가 포함된 댓글입니다.'

In [47]:
loaded_model = load_model('./best_model.h5')

## 크롤링 데이터

In [ ]:
# forders = os.listdir('/home/adminuser/notebooks/modeling/raw_data/crawling/')

# df_all = pd.DataFrame()

# for i in range(0,len(forders)):
#     if forders[i].split('.')[1] == 'csv':
#         file = '/home/adminuser/notebooks/modeling/raw_data/crawling/'+forders[i]
#         df= pd.read_csv(file,encoding='utf-8') 
#         df_all = pd.concat([df_all, df])

# df_all = df_all.dropna()

In [120]:
# 크롤링 데이터셋 합치기

import os
file_list= os.listdir("../raw_data/crawling")
base= "../raw_data/crawling/"
prev=pd.DataFrame()
for i in file_list[2:4]: #마지막 5개 데이터만 합치기
    print(base+i)
    df= pd.read_csv(base+i)
    prev= pd.concat([prev,df], axis=0)

../raw_data/crawling/네이쳐러브메레❄️Cool~❄️리츄얼낮잠이불54%OFF.csv
../raw_data/crawling/가정의_달_맞이❤보오글_시그니처_에디션_오픈🎁.csv


In [130]:
# def sentiment_predict(new_sentence):
#     new_sentence = re.compile('[^ A-Za-z0-9ㄱ-ㅣ가-힣]+').sub('', str(new_sentence))
#     new_sentence = okt.morphs(new_sentence, stem=True) # 토큰화        
#     new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거
#     encoded = tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩
#     pad_new = pad_sequences(encoded, maxlen = max_len) # 패딩
#     score = float(load_model).predict(pad_new) # 예측
#     if(score > 0.3): # 1에 가까워질수록 욕의 범위가 넓어짐, 0일때 필터링 강도가 최대
#         return "{:.2f}% 확률로 비속어가 포함된 댓글입니다.".format(score * 100)
#     else:
#         return "{:.2f}% 확률로 일반 댓글입니다.".format((1 - score) * 100)

In [139]:
def sentiment_predict(new_sentence):
    new_sentence = re.compile('[^ A-Za-z0-9ㄱ-ㅣ가-힣]+').sub('', str(new_sentence))
    new_sentence = okt.morphs(new_sentence, stem=True) # 토큰화        
    new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거
    encoded = tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩
    pad_new = pad_sequences(encoded, maxlen = max_len) # 패딩
    score = float(loaded_model.predict(pad_new)) # 예측
    if(score > 0.4): # 1에 가까워질수록 필터링강도가 약해지고, 0에 가까울때 필터링 강도가 최대
        return "{:.2f}% 확률로 비속어가 포함된 댓글입니다.".format(score * 100)
    else:
        return "{:.2f}% 확률로 일반 댓글입니다.".format((1 - score) * 100)

In [140]:
sentiment_predict('배고파') #0.8 을 기준으로 비속어와 안 비속어 갈라짐

'57.75% 확률로 비속어가 포함된 댓글입니다.'

In [141]:
sentiment_predict('하지말라고 좀') #0.6 일반 0.4 비속어

'57.75% 확률로 비속어가 포함된 댓글입니다.'

In [135]:
sentiment_predict('가나가나')

'57.75% 확률로 비속어가 포함된 댓글입니다.'

In [136]:
sentiment_predict('밥 먹어야지')

'68.58% 확률로 일반 댓글입니다.'

In [137]:
sentiment_predict('ㄱㄱ')

'57.75% 확률로 비속어가 포함된 댓글입니다.'

In [143]:
prev['result'] = prev[["comment"]].comment.map(lambda x: sentiment_predict(x))

In [137]:
if '*비속어' in prev['result']:
    print(prev['comment'])
else:
    pass

In [144]:
def check_a(x):
    if "비속어" in x:
        return 
prev[['comment', 'result']]

,comment,result
0,우와!!,57.75% 확률로 비속어가 포함된 댓글입니다.
1,어머.. 진짜 뜯어서 보여주시니까 바로 믿을수 있을것 같아요!!!!,90.57% 확률로 일반 댓글입니다.
2,여름에 쓰기 좋겠어요!!,71.29% 확률로 일반 댓글입니다.
3,헐..ㅜㅜ 특가는 또 안하나요?ㅜㅜ,61.05% 확률로 일반 댓글입니다.
4,카시트에도 설치가능한거 볼 수 있나요?,97.12% 확률로 일반 댓글입니다.
...,...,...
116,투명유리,57.75% 확률로 비속어가 포함된 댓글입니다.
117,사용기간은 어느정도 나요?,93.53% 확률로 일반 댓글입니다.
118,코팅 수명 끝나면 다 교환해쥼,71.50% 확률로 일반 댓글입니다.
119,잠만요 옵션 보고 온다 ㅋㅋㅋㅋ,94.75% 확률로 일반 댓글입니다.


In [140]:
def check_a(x):
    if "비속어" in x:
        return 

In [151]:
prev[['comment','result']].sample(10)

,comment,result
18,ㅋㅋ 주방 용품 계 꾸안꾸,61.33% 확률로 일반 댓글입니다.
101,뚜껑도 좋네요,72.01% 확률로 일반 댓글입니다.
111,뚜껑 넘 좋아보이네요,85.95% 확률로 일반 댓글입니다.
49,바쁘다 바쁜 난주언니ㅋㅋ,79.42% 확률로 일반 댓글입니다.
42,ㅋㅋㅋ,57.75% 확률로 비속어가 포함된 댓글입니다.
4,있어보여요 ㅎ,94.33% 확률로 일반 댓글입니다.
99,맢치마도 버거요 ㅋ,90.30% 확률로 일반 댓글입니다.
5,바다로 갑니다,88.33% 확률로 일반 댓글입니다.
22,가 아니고 ~,98.18% 확률로 일반 댓글입니다.
37,추카드려요♡,76.13% 확률로 일반 댓글입니다.


In [183]:
def slang_predict(new_sentence):
    new_sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', '', new_sentence)
    if new_sentence == "" or new_sentence.isspace():
        return "일반 댓글"
    else : 
        new_sentence = okt.morphs(new_sentence, stem=True)
        new_sentence = [word for word in new_sentence if not word in stopwords]
        if not new_sentence:
            return "일반 댓글"
        else:
            encoded = tokenizer.texts_to_sequences([new_sentence])
            pad_new = pad_sequences(encoded, maxlen = max_len)
            score = loaded_model.predict(pad_new)
            if(score > 0.4): # 1에 가까워질수록 필터링강도가 약해지고, 0에 가까울때 필터링 강도가 최대
                return "비속어 포함"
            else:
                return "일반 댓글"
            
        
#         return round(float(score), 2)
        
#     if(score > 0.4): # 1에 가까워질수록 필터링강도가 약해지고, 0에 가까울때 필터링 강도가 최대
#         return "{:.2f}% 확률로 비속어가 포함된 댓글입니다.".format(score * 100)
#     else:
#         return "{:.2f}% 확률로 일반 댓글입니다.".format((1 - score) * 100)

In [184]:
prev['result2'] = prev[["comment"]].comment.map(lambda x: slang_predict(x))

In [216]:
prev[['comment','result']].sample(10)

,comment,result
50,낮잠이불 디자인 진짜 이쁘다,90.39% 확률로 일반 댓글입니다.
64,블랙도 너무 이쁘네요,88.93% 확률로 일반 댓글입니다.
33,바닥에 깔린상품은 뭐에요??,85.76% 확률로 일반 댓글입니다.
57,분리형 일체형,66.31% 확률로 일반 댓글입니다.
31,ㅋㅋㅋㅋㅋㅋㅋㅋ,57.75% 확률로 비속어가 포함된 댓글입니다.
27,계곡물에 수박 넣는 느낌,44.00% 확률로 비속어가 포함된 댓글입니다.
31,1+1이네요ㅎㅎㅎ 역시 네러메 라방은 혜자네요~,82.01% 확률로 일반 댓글입니다.
84,제품 기대됩니다,92.63% 확률로 일반 댓글입니다.
23,휴대용 유모차 쓸랴고 하는데 배게도 한번 붙야주세요융,86.72% 확률로 일반 댓글입니다.
116,투명유리,57.75% 확률로 비속어가 포함된 댓글입니다.


In [193]:
def swear_predict(new_sentence):
    new_sentence = re.compile('[^ 0-9ㄱ-ㅎㅏ-ㅣ가-힣]+').sub('', str(new_sentence))
    if new_sentence == "" or new_sentence.isspace():
        return "일반 댓글"
    else:
        new_sentence = okt.morphs(new_sentence, stem=True) # 토큰화
        new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거
        if not new_sentence:
            return "일반 댓글"
        else:    
            encoded = tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩
            pad_new = pad_sequences(encoded, maxlen = max_len) # 패딩
            score = float(loaded_model.predict(pad_new)) # 예측
            if(score > 0.4):
                print(new_sentence)
                print("{:.2f}% 확률로 비속어가 포함된 댓글입니다.\n".format(score * 100))
                #  return round(float(score), 3)
            else:
                print(new_sentence)
                print("{:.2f}% 확률로 일반 댓글입니다.\n".format((1 - score) * 100))

In [194]:
prev['result4'] = prev[["comment"]].comment.map(lambda x: swear_predict(x))

['우와']
57.75% 확률로 비속어가 포함된 댓글입니다.

['어머', '진짜', '뜯다', '보여주다', '바로', '믿다', '있다', '같다']
90.57% 확률로 일반 댓글입니다.

['여름', '쓰기', '좋다']
71.29% 확률로 일반 댓글입니다.

['헐다', 'ㅜㅜ', '특가', '또', '알다', 'ㅜㅜ']
61.05% 확률로 일반 댓글입니다.

['카시트', '에도', '설치', '가능하다', '볼', '수', '있다']
97.12% 확률로 일반 댓글입니다.

['저렇게', '테스트', '까지', '아이', '여름', '쉬', '원하다', '같다']
81.93% 확률로 일반 댓글입니다.

['시원하다']
45.56% 확률로 비속어가 포함된 댓글입니다.

['통', '기성', '좋다']
66.62% 확률로 일반 댓글입니다.

['대박', '대박']
57.75% 확률로 비속어가 포함된 댓글입니다.

['구매', '습', '니당다']
44.29% 확률로 비속어가 포함된 댓글입니다.

['맘', '님', '들다', '혹시', '신생아', '때', '써다', '되다']
95.33% 확률로 일반 댓글입니다.

['여름', '아', '기다리다', '우린', '준비', '돼다']
82.03% 확률로 일반 댓글입니다.

['저', '선풍기', '색깔', '너무', '예쁘다']
91.16% 확률로 일반 댓글입니다.

['오', '쿨', '라이너', '정말', '시원하다', '같다', '대박', 'ㅎㅎㅎㅎ']
88.41% 확률로 일반 댓글입니다.

['어디', '든', '호환', '가능하다']
77.53% 확률로 일반 댓글입니다.

['살랑살랑']
57.75% 확률로 비속어가 포함된 댓글입니다.

['라이너', '대박', '이네', '여']
64.86% 확률로 일반 댓글입니다.

['우리', '아들', '열', '많다', '걱정', '이다', '너무', '좋아하다']
97.89% 확률로 일반 댓글입니다.

['감사']
57.75% 확률로 비속어가 포함된 댓

In [195]:
prev[['comment','result','result2', 'result3', 'result4']].sample(10)

,comment,result,result2,result3,result4
56,퀴즈 쉬운걸로,95.22% 확률로 일반 댓글입니다.,일반 댓글,일반 댓글,None
109,투명하다ㅋ,57.75% 확률로 비속어가 포함된 댓글입니다.,비속어 포함,비속어,None
58,부탁합니다,60.06% 확률로 일반 댓글입니다.,일반 댓글,일반 댓글,None
8,대박대박,57.75% 확률로 비속어가 포함된 댓글입니다.,비속어 포함,비속어,None
40,축하드려요,79.08% 확률로 일반 댓글입니다.,일반 댓글,일반 댓글,None
55,The Love..,57.75% 확률로 비속어가 포함된 댓글입니다.,일반 댓글,0,일반 댓글
2,여름에 쓰기 좋겠어요!!,71.29% 확률로 일반 댓글입니다.,일반 댓글,일반 댓글,None
102,비교해주시 이해가 가네요,88.99% 확률로 일반 댓글입니다.,일반 댓글,일반 댓글,None
92,앞치마도 진짜 이뻐요♡,90.33% 확률로 일반 댓글입니다.,일반 댓글,일반 댓글,None
6,고급졍,63.98% 확률로 일반 댓글입니다.,일반 댓글,일반 댓글,None
